In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../"))
from controlunit.readsettings import init_configuration, load_settings

In [2]:
config = init_configuration('../controlunit/settings.yml')

I'll try to create datafolder: C:\Users\queez\work\cudata
Already exists.
controlunit configuration loaded successfully.


In [3]:
print(config['Data Folder'])

C:\Users\queez\work\cudata


In [4]:
config['ADC Additional Columns']

['date', 'time', 'IGmode', 'IGscale', 'QMS_signal']

In [5]:
config['ADC Column Names']

['date',
 'time',
 'IGmode',
 'IGscale',
 'QMS_signal',
 'Pd',
 'Pu',
 'Ip',
 'Bu',
 'Bd',
 'Pd_c',
 'Pu_c',
 'Ip_c',
 'Bu_c',
 'Bd_c']

In [6]:
config['ADC Signal Names']

['Pd', 'Pu', 'Ip', 'Bu', 'Bd']

In [7]:
config['ADC Converted Names']

['Pd_c', 'Pu_c', 'Ip_c', 'Bu_c', 'Bd_c']

In [8]:
config['ADC Additional Columns']

['date', 'time', 'IGmode', 'IGscale', 'QMS_signal']

In [9]:
config['LED GPIO']

27

In [10]:
pres = config['Adc Channel Properties']['Pd']

In [11]:
pres.conversion.__name__

'ionization_gauge'

In [14]:
len(config['ADC Column Names'])

15

In [12]:
for i,j in config['Adc Channel Properties'].items():
    print(j.channel)
    
    

0
1
2
3
4
